Run TDE/nuclear program manually.

In [ ]:
import requests, os
from astropy.time import Time

In [ ]:
token = os.environ["ARCHIVE_TOKEN"]
header = {"Authorization": "bearer "+token}
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.ztfbh.t0.NuclearFilter import NuclearFilter

from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
logger = AmpelLogger.get_logger()

## A. Define filter parameters
Copied from latest release.

In [ ]:
# Decent filter parameters
filter_config = {
     "minDetections": 3,
     "maxDistPS1source": 0.5,
     "closePS1dist": 0.5,
     "maxDeltaRad": 0.5,
     "diffmagLimit": 20,
     "maxDeltaMag": 2.5,
    "resource": {"ampel-ztf/catalogmatch":"https://ampel.zeuthen.desy.de/api/catalogmatch/"},
}

In [ ]:
t0filter = NuclearFilter( **filter_config, logger=logger )
t0filter.post_init()

## B. Getting a handle to alerts.
A _resume token_, which points to a specific set of alerts. We limit the alerts we download through an initial query based on direct alert properties, where possible. 

In [ ]:
delta_t = 3.

In [ ]:
# Adding some handles to make it feasible...
query = {
  "jd": {
    "$gt": Time.now().jd-delta_t,
    "$lt": Time.now().jd
  },
  "candidate": {
    "distpsnr1": {
      "$lt": filter_config['maxDistPS1source']
    },
#    "drb": {
#      "$gt": 0.95   
#    },
    "ndethist": {
      "$gte": filter_config['minDetections'],
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
# Connection
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?'
header = {"Authorization": "bearer "+os.environ["ARCHIVE_TOKEN"]}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
response.ok

In [ ]:
resume_token = response.json()['resume_token']

## C. Define the database and channel parameters

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel83/Ampel-HU-astro/ampel_conf.yaml'

In [ ]:
channel = 'NuclearNB'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'Nuclear',
    purge_db = False,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB']
)

In [ ]:
# Will use NED for spectroscopic redshifts.
cat_conf = {
    'catalogs' : {
        'SDSS_spec' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.,
            'keys_to_append' : ['z', 'bptclass', 'subclass'],
            'all': False,
        },
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 2.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'GLADEv23' : {
            'use' : 'extcats',
            'rs_arcsec' : 2,
            'keys_to_append' : ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },
         'PS1_photoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.0,
            'keys_to_append' : ['raMean','decMean','z_phot','z_photErr', 'z_phot0', '_2'],
            'pre_filter' : None,
            'post_filter' : None,
        },
         'PS1' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 1,
        },
         'brescia' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.,
            'keys_to_append' : [],
        },
         'milliquas' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.,
            'keys_to_append' : ['broad_type','ref_name'],
        },
         'portsmouth' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.,
            'keys_to_append' : ['sigma_stars','sigma_stars_err','bpt'],
        },
         'ptfvar' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.,
            'keys_to_append' : [],
        },
         'varstars' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.,
            'keys_to_append' : [],
        },
         'wise_color' : {
            'use' : 'extcats',
            'rs_arcsec' : 2.,
            'keys_to_append' : ['W1mW2'],
        },
    }
}

In [ ]:
# New, new, also with all categories?
flexfit_conf = {
    "oldest_upper_limits" : 14,
    "max_post_peak" : 200,
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "NuclearFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2FlexFit",
                                    "config": flexfit_conf,
                                },
                                {
                                    "unit": "T2SimpleMetrics",
                                },
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                        ],
                    }

                }
            }
        }
    ]

In [ ]:
response.json()

In [ ]:
loader_config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":response.json()['resume_token']}

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = 1000000000,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': None,
            'loader': {
                'unit': 'ZTFArchiveAlertLoader',
                'config': loader_config
            }
        }
    },
    directives = directives
)


## D. Process alerts. These will be downloaded from the archive.

In [ ]:
n = ac.run()

In [ ]:
print('Processed {} alerts locally (based on query output).'.format(n))

## E. Do T2 calculations

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

## F. Run T3s...